In [1]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

In [2]:
texts = [
    "LangChain est une composante clé dans la création d'applications avec des LLM.",
    "La méthode BM25 permet d'améliorer la qualité de la recherche documentaire.",
    "Avec FAISS, les résultats sont plus pertinents grâce à l'analyse sémantique.",
    "Le retrieval hybride est souvent combiné avec des approches lexicales pour des performances accrues.",
    "L'intégration de embeddings dans un pipeline RAG est désormais plus simple grâce à LangChain.",
    "Les systèmes de vector search utilisent des embeddings pour représenter le texte numériquement.",
    "FAISS améliore la précision de la recherche en comprenant le sens des requêtes.",
    "BM25 fonctionne très bien en local, même sans infrastructure cloud.",
    "Grâce à LangChain, on peut interroger efficacement une base documentaire complexe.",
    "La combinaison de BM25 et FAISS est très utilisée dans les systèmes RAG modernes.",
    "LLM est une composante clé dans la création d'applications avec des LLM.",
    "La méthode hybrid methods permet d'améliorer la qualité de la recherche documentaire.",
    "Avec dense retrieval, les résultats sont plus pertinents grâce à l'analyse sémantique.",
    "vector search est souvent combiné avec des approches lexicales pour des performances accrues.",
    "L'intégration de retrieval dans un pipeline RAG est désormais plus simple grâce à LangChain.",
    "Les systèmes de metadata utilisent des embeddings pour représenter le texte numériquement.",
    "question answering améliore la précision de la recherche en comprenant le sens des requêtes.",
    "open source fonctionne très bien en local, même sans infrastructure cloud.",
    "Grâce à document processing, on peut interroger efficacement une base documentaire complexe.",
    "La combinaison de FAISS et embeddings est très utilisée dans les systèmes RAG modernes.",
    "LangChain est souvent combiné avec FAISS dans les pipelines IA modernes.",
    "BM25 est toujours utilisé dans les moteurs de recherche traditionnels.",
    "Les embeddings permettent une compréhension sémantique fine.",
    "Une base de connaissance bien structurée améliore le retrieval.",
    "Chunker les documents aide les retrievers à mieux cibler les informations.",
    "Les LLMs comme Mistral ou LLaMA profitent d’un bon contexte via retrieval.",
    "Le vector store est essentiel pour la recherche dense.",
    "FAISS et Chroma sont les solutions les plus courantes en local.",
    "LangChain propose une API unifiée pour tous les types de retrievers.",
    "Le format des documents (PDF, Markdown, HTML) influence le chunking.",
    "BM25 reste performant sur les corpus simples ou très structurés.",
    "Le reranking améliore la qualité finale des documents proposés au LLM.",
    "Certaines approches hybrides utilisent des pondérations dynamiques.",
    "Weaviate permet de combiner recherche vectorielle et filtrage structuré.",
    "Avec Ollama, les LLMs peuvent tourner localement sans cloud.",
    "Le coût des embeddings dépend du modèle utilisé (MiniLM, MPNet…).",
    "La qualité du prompt est influencée par le contexte fourni par le retriever.",
    "Un bon split des documents augmente le recall.",
    "Les métriques comme le MRR ou Recall@k permettent d’évaluer un retriever.",
    "L’infrastructure locale doit être optimisée pour charger rapidement l’index.",
    "L’ajout de métadonnées enrichit le filtrage contextuel dans les retrievers.",
    "LangChain permet de tester rapidement plusieurs stratégies de retrieval.",
    "Une pipeline RAG efficace passe par un tuning précis du retriever.",
    "Le choix de l’embedding model impacte directement la pertinence des résultats.",
    "Il est utile d’indexer les sources des documents pour l’auditabilité.",
    "Certaines approches utilisent des embeddings multi-vecteurs par chunk.",
    "FAISS HNSW est plus performant pour des recherches approximatives rapides.",
    "La vectorisation peut se faire offline pour gagner en performance.",
    "LangChain offre aussi un support pour OpenSearch et Qdrant.",
    "Un LLM avec peu de contexte aura du mal à répondre précisément.",
    "Il faut ajuster la taille et le recouvrement des chunks pour chaque cas d’usage."
]
documents = [Document(page_content=t) for t in texts]

In [3]:
splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs_split = splitter.split_documents(documents)

In [4]:
embedding_models = {
    "miniLM": HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    "mpnet": HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
}

In [8]:
retrievers = {}

for name, embedding in embedding_models.items():
    vectorstore = FAISS.from_documents(docs_split, embedding)
    retrievers[f"FAISS ({name})"] = vectorstore.as_retriever(search_kwargs={"k": 3})

bm25_retriever = BM25Retriever.from_documents(docs_split)
bm25_retriever.k = 3
retrievers["BM25"] = bm25_retriever

hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retrievers["FAISS (miniLM)"]],
    weights=[0.5, 0.5]
)
retrievers["Hybrid (BM25 + FAISS-miniLM)"] = hybrid_retriever

In [11]:
def test_query(query: str, top_k: int = 2):
    print(f"\n🔍 Question : {query}")
    for name, retriever in retrievers.items():
        print(f"\n➡ Méthode : {name}")
        results = retriever.invoke(query)
        for i, doc in enumerate(results):
            print(f"{i+1}. {doc.page_content}")
    print("\n" + "="*50)

In [12]:
test_query("Comment fonctionne la recherche vectorielle ?")
test_query("Qu'est-ce qu'un modèle d'embedding ?")


🔍 Question : Comment fonctionne la recherche vectorielle ?

➡ Méthode : FAISS (miniLM)
1. Le vector store est essentiel pour la recherche dense.
2. Weaviate permet de combiner recherche vectorielle et filtrage structuré.
3. La vectorisation peut se faire offline pour gagner en performance.

➡ Méthode : FAISS (mpnet)
1. La vectorisation peut se faire offline pour gagner en performance.
2. Weaviate permet de combiner recherche vectorielle et filtrage structuré.
3. Le vector store est essentiel pour la recherche dense.

➡ Méthode : BM25
1. Weaviate permet de combiner recherche vectorielle et filtrage structuré.
2. La méthode BM25 permet d'améliorer la qualité de la recherche documentaire.
3. La méthode hybrid methods permet d'améliorer la qualité de la recherche documentaire.

➡ Méthode : Hybrid (BM25 + FAISS-miniLM)
1. Weaviate permet de combiner recherche vectorielle et filtrage structuré.
2. Le vector store est essentiel pour la recherche dense.
3. La méthode BM25 permet d'améliorer l